In [29]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import json
import networkx as nx
from datetime import datetime
import sys
import csv

In [2]:
!head 'data/08.18.2021.14.10.34EDGELIST.csv'
!head 'data/08.17.2021.21.49.40LABELS.csv'

Source,Target,Quantity,Tradeskill,Rarity,Station,ItemType,Category
workorder_workrationst5,workrationst5,0,Cooking,0,Kitchen Tier 5,Resource,Work Orders
workorder_workrationst4,workrationst4,0,Cooking,0,Kitchen Tier 4,Resource,Work Orders
workorder_workrationst3,workrationst3,0,Cooking,0,Kitchen Tier 3,Resource,Work Orders
workorder_workrationst2,workrationst2,0,Cooking,0,Kitchen Tier 2,Resource,Work Orders
reagentconvertert5,tannint5,2,Weaving,2,Loom Tier 5,Resource,Material Conversion
Refining Materials Tier 5,tannint5,0,Weaving,2,Loom Tier 5,Resource,Material Conversion
reagentconvertert4,tannint4,1,Weaving,1,Loom Tier 4,Resource,Material Conversion
Refining Materials Tier 4,tannint4,0,Weaving,1,Loom Tier 4,Resource,Material Conversion
reagentconvertert3,tannint3,0,Weaving,0,Loom Tier 3,Resource,Material Conversion
ID,Label,Rarity
workorder_workrationst5,Hearty Worker Rations,0
workorder_workrationst4,Large Worker Rations,0
workorder_workrationst3,Small Worker Rations,0
workorder_wo

In [3]:
edge_file = 'data/08.18.2021.14.10.34EDGELIST.csv'
label_file = 'data/08.17.2021.21.49.40LABELS.csv'

edgeData = open(edge_file, "r")
next(edgeData, None)  # skip the first line in the input file

labelData = open(label_file, "r")
next(labelData, None)  # skip the first line in the input file
edgeGraph = nx.DiGraph()
labelGraph = nx.DiGraph()

G = nx.parse_edgelist(edgeData, delimiter=',', create_using=edgeGraph,
                      nodetype=str, data=(('Quantity', int),
                                          ('Tradeskill', str),
                                          ('Rarity', int),
                                          ('Station', str),
                                          ('ItemType', str),
                                          ('Category', str),))

In [4]:
NG = nx.DiGraph()
nodes = pd.read_csv(label_file)
format_nodes = [(item[0], {'name': item[1], 'Rarity': item[2]})for item in nodes.values.tolist()]
NG.add_nodes_from(format_nodes)

# NG = nx.parse_adjlist(labelData, delimiter=',', create_using=labelGraph, nodetype=str)

In [5]:
F = nx.compose(NG,G)

In [6]:
print("G nodes {} NG nodes {} F nodes {}".format(len(list(G)),len(list(NG)),len(list(F)),))

G nodes 3497 NG nodes 1228 F nodes 3529


In [7]:
end_items = sorted([ingId for ingId, degree in F.out_degree() if degree == 0 ])
print(len(end_items))
start_items = sorted([ingId for ingId, degree in F.in_degree() if degree == 0 ])
print(len(start_items))

2341
960



Make sure all nodes have a rarity since we use it for baseline value.

Remove cycles to make things easier. They're just going up and down refined materials anyways.

Initialize all items with no predecessors with a value dependant on their rarity

In [8]:
def get_digits(text):
    return list(filter(str.isdigit, text))

def clean_node_rarities(F, rarity_values):
    
    for n,d in F.nodes(data=True):
        digits = get_digits(n)
        if len(digits) == 1:
            r = digits[0]
        else:
            r = 0
        F.node[n]['Rarity'] = r
    
    for s,t,d in F.edges(data=True):
        r = d['Rarity']
        q = d['Quantity']
        if q == 0:
            F.edges[s,t]['Quantity'] = 1
            q = 1
        v = rarity_values[r] * q
#         print("Source: {} Target: {} Value: {} * {} = {}".format(s,t,rarity_values[r], q, v))
        F.node[t]['Rarity'] = r

def remove_cycles(F):
    for cycle in nx.algorithms.cycles.simple_cycles(F):
        source = [int(s) for s in cycle[0] if s.isdigit()]
        if len(cycle) > 1:
            sink = [int(s) for s in cycle[1] if s.isdigit()]
            F.remove_edge(cycle[0], cycle[1])
        else:
            F.remove_edge(cycle[0], cycle[0])

    for cycle in nx.algorithms.cycles.simple_cycles(F):
        print("Cycle left:", cycle)  
        
        
def initialize_start_items(F, start_items, values, rarity_values):
    for ingId in start_items:
        data = F.node[ingId]
        rarity = data['Rarity']
        value = rarity_values[int(rarity)]
        values[ingId] = value
        F.node[ingId]['Value'] = value
        
def reset(edge_values, node_values, rarity_values, F):
    clean_node_rarities(F, rarity_values)
    remove_cycles(F)
    initialize_start_items(F, start_items, node_values, rarity_values)

In [9]:
edge_updates = 0
node_updates = 0

def add_edge_value(s, t, ev, edge_values, F):
    global edge_updates
    edge_updates = edge_updates + 1
    F.edges[s,t]['Value'] = ev
    edge_values[(s,t)] = ev
    
def add_node_value(node, value, node_values, F):
    global node_updates
    node_updates = node_updates + 1
    F.node[node]['Value'] = value
    node_values[node] = value

def compute_node_value_smol(node, node_values, edge_values, F):
    if node in node_values:
        print("Using existing node value for {}: {}".format(node, node_values[node]))
        return node_values[node]
    else:
        edge_list = []
        preds = list(F.predecessors(node))
        if len(preds) == 0:
            print("Using len(preds) == 0 logic for value.")
            node_value = rarity_values[F.node[node]['Rarity']]
            edge_list = [node_value]
        else:
            for p in preds:
                ev = compute_edge_value_smol(p, node, node_values, edge_values, F)
                edge_list.append(ev)
                node_value = sum(edge_list)
        print("New node value for {} is {}".format(node, node_value, edge_list))
        add_node_value(node, node_value, node_values, F)
        return node_value
        
    
def compute_edge_value_smol(s, t, node_values, edge_values, F):
    """
    Recipe ingredients are sources for the target recipe output.
    F: The graph
    s: source node 
    t: target node
    """
    if (s,t) in edge_values:
        print("Using existing edge value for {}: {}".format((s,t), edge_values[(s,t)]))
        return edge_values[(s,t)]
    s_value = compute_node_value_smol(s, node_values, edge_values, F)
    ev = s_value * F.edges[s,t]['Quantity']
    print("New edge value for {} is {}".format((s,t), ev))
    add_edge_value(s, t, ev, edge_values, F)
    return ev

edge_values = {}
node_values = {}
base = 3
rng = 6
rarity_values = [base ** i for i in range(0,rng)]
print(rarity_values)
reset(edge_values, node_values, rarity_values, F)
# print(edge_values, node_values.values())
for node_id in F.nodes():
    print(node_id)
    compute_node_value_smol(node_id, node_values, edge_values, F) 

print("Edge Updates: {} \n Node Updates: {}".format(edge_updates, node_updates))
# iid = '2hgreataxe_darkskyt5'
# # iid = 'house_housingitem_buff_skinning_bonus_t2'
# data = F.node[iid]
# if iid in node_values:
#     del node_values[iid]
# compute_node_value_smol(iid, node_values, edge_values, F)

[1, 3, 9, 27, 81, 243]
workorder_workrationst5
Using existing node value for workorder_workrationst5: 243
workorder_workrationst4
Using existing node value for workorder_workrationst4: 81
workorder_workrationst3
Using existing node value for workorder_workrationst3: 27
workorder_workrationst2
Using existing node value for workorder_workrationst2: 9
reagentconvertert5
Using existing node value for reagentconvertert5: 243
Refining Materials Tier 5
Using existing node value for Refining Materials Tier 5: 243
reagentconvertert4
Using existing node value for reagentconvertert4: 81
Refining Materials Tier 4
Using existing node value for Refining Materials Tier 4: 81
reagentconvertert3
Using existing node value for reagentconvertert3: 27
Refining Materials Tier 3
Using existing node value for Refining Materials Tier 3: 27
solventt5
Using existing node value for reagentconvertert5: 243
New edge value for ('reagentconvertert5', 'solventt5') is 486
Using existing node value for Refining Material

Using existing node value for reagentconvertert5: 243
New edge value for ('reagentconvertert5', 'clothweavet5') is 486
Using existing node value for Refining Materials Tier 5: 243
New edge value for ('Refining Materials Tier 5', 'clothweavet5') is 243
New node value for clothweavet5 is 729
New edge value for ('clothweavet5', 'clotht51') is 1458
Using existing node value for clotht5: 381
New edge value for ('clotht5', 'clotht51') is 381
Using existing node value for scaleclotht1: 3
New edge value for ('scaleclotht1', 'clotht51') is 12
Using existing node value for blisterweavet1: 3
New edge value for ('blisterweavet1', 'clotht51') is 12
New node value for clotht51 is 1863
clawt5
Using existing node value for clawt5: 243
voidbentingot
Using existing node value for voidbentingot: 1
fangt4
Using existing node value for fangt4: 81
scarhidet1
Using existing node value for scarhidet1: 3
azoth_currency
Using existing node value for azoth_currency: 1
syndicateseal_occultist
Using existing node 

New edge value for ('alchemydeatht1', 'house_housingitem_buff_damage_vs_lost_t1') is 3
New node value for house_housingitem_buff_damage_vs_lost_t1 is 74
New edge value for ('house_housingitem_buff_damage_vs_lost_t1', 'house_housingitem_buff_damage_vs_lost_t2') is 74
New node value for house_housingitem_buff_damage_vs_lost_t2 is 594
ectoplasmicessencet1
Using existing node value for ectoplasmicessencet1: 3
house_housingitem_buff_damage_vs_lost_t1
Using existing node value for house_housingitem_buff_damage_vs_lost_t1: 74
corruptedtotemt1
Using existing node value for corruptedtotemt1: 3
house_housingitem_buff_damage_vs_corrupted_t2
Using existing node value for woodstaint4: 165
New edge value for ('woodstaint4', 'house_housingitem_buff_damage_vs_corrupted_t2') is 330
Using existing node value for timbert4: 92
New edge value for ('timbert4', 'house_housingitem_buff_damage_vs_corrupted_t2') is 92
Using existing node value for ingott4: 89
New edge value for ('ingott4', 'house_housingitem_bu

pigmentbrown
Using existing node value for pigmentbrown: 1
pigmentcyan
Using existing node value for pigmentcyan: 1
pigmentturquoise
Using existing node value for pigmentturquoise: 1
pigmentorange
Using existing node value for pigmentorange: 1
workorder_dispellingkitt5
Using existing node value for workorder_dispellingkitt5: 243
workorder_dispellingkitt4
Using existing node value for workorder_dispellingkitt4: 81
workorder_dispellingkitt3
Using existing node value for workorder_dispellingkitt3: 27
workorder_dispellingkitt2
Using existing node value for workorder_dispellingkitt2: 9
diamondt3
Using existing node value for diamondt3: 14
diamondt2
Using existing node value for diamondt2: 7
diamondt1
Using existing node value for diamondt1: 3
diamondt4
Using existing node value for diamondt4: 47
dexstrbonust3
Using existing node value for transmutationcatalystt4: 81
New edge value for ('transmutationcatalystt4', 'dexstrbonust3') is 162
Using existing node value for transmutationcatalystt3: 

New edge value for ('Trinket Bands', 'ringtopazt4') is 1
Using existing node value for Trinket Gem Settings: 1
New edge value for ('Trinket Gem Settings', 'ringtopazt4') is 1
Using existing node value for Precious Metals: 1
New edge value for ('Precious Metals', 'ringtopazt4') is 1
New node value for ringtopazt4 is 73
ringtopazt3
Using existing node value for topazcutt2: 11
New edge value for ('topazcutt2', 'ringtopazt3') is 11
Using existing node value for Trinket Bands: 1
New edge value for ('Trinket Bands', 'ringtopazt3') is 1
Using existing node value for Trinket Gem Settings: 1
New edge value for ('Trinket Gem Settings', 'ringtopazt3') is 1
Using existing node value for Precious Metals: 1
New edge value for ('Precious Metals', 'ringtopazt3') is 1
New node value for ringtopazt3 is 14
ringtopazt2
Using existing node value for Trinket Bands: 1
New edge value for ('Trinket Bands', 'ringtopazt2') is 1
Using existing node value for Trinket Gem Settings: 1
New edge value for ('Trinket Ge

...

heavychest_furlinedt5
Using existing node value for Tier 5 Metals: 243
New edge value for ('Tier 5 Metals', 'heavychest_furlinedt5') is 243
Using existing node value for Leathers: 1
New edge value for ('Leathers', 'heavychest_furlinedt5') is 1
Using existing node value for Cloth: 1
New edge value for ('Cloth', 'heavychest_furlinedt5') is 1
New node value for heavychest_furlinedt5 is 245
heavychest_crafteddungeon7t5
Using existing node value for Tier 5 Metals: 243
New edge value for ('Tier 5 Metals', 'heavychest_crafteddungeon7t5') is 243
Using existing node value for Leathers: 1
New edge value for ('Leathers', 'heavychest_crafteddungeon7t5') is 1
Using existing node value for Cloth: 1
New edge value for ('Cloth', 'heavychest_crafteddungeon7t5') is 1
Using existing node value for stickyvinest1: 3
New edge value for ('stickyvinest1', 'heavychest_crafteddungeon7t5') is 3
New node value for heavychest_crafteddungeon7t5 is 248
heavychest_crafteddungeon6t5
Using existing node value for Tier 

New edge value for ('peppercornt1', 'fooddext2') is 3
Using existing node value for meatgamet1: 3
New edge value for ('meatgamet1', 'fooddext2') is 3
New node value for fooddext2 is 16
fooddexstrt5
Using existing node value for carrott1: 3
New edge value for ('carrott1', 'fooddexstrt5') is 3
Using existing node value for potatot1: 3
New edge value for ('potatot1', 'fooddexstrt5') is 3
Using existing node value for parsleyt1: 3
New edge value for ('parsleyt1', 'fooddexstrt5') is 3
Using existing node value for saget1: 3
New edge value for ('saget1', 'fooddexstrt5') is 3
Using existing node value for thymet1: 3
New edge value for ('thymet1', 'fooddexstrt5') is 3
Using existing node value for cranberryt1: 3
New edge value for ('cranberryt1', 'fooddexstrt5') is 3
Using existing node value for turkeythight1: 3
New edge value for ('turkeythight1', 'fooddexstrt5') is 3
New node value for fooddexstrt5 is 21
fooddexstrt4
Using existing node value for eggt1: 3
New edge value for ('eggt1', 'foodd

Using existing node value for workorder_faction_armaments09_t3: 1
New edge value for ('workorder_faction_armaments09_t3', 'faction_armaments09_t3') is 1
New node value for faction_armaments09_t3 is 177
faction_armaments09_t2
Using existing node value for oret1: 3
New edge value for ('oret1', 'faction_armaments09_t2') is 3
Using existing node value for rawhidet1: 3
New edge value for ('rawhidet1', 'faction_armaments09_t2') is 3
Using existing node value for woodt1: 3
New edge value for ('woodt1', 'faction_armaments09_t2') is 3
Using existing node value for workorder_faction_armaments09_t2: 1
New edge value for ('workorder_faction_armaments09_t2', 'faction_armaments09_t2') is 1
New node value for faction_armaments09_t2 is 10
faction_armaments08_t5
Using existing node value for timbert4: 92
New edge value for ('timbert4', 'faction_armaments08_t5') is 92
Using existing node value for ingott4: 89
New edge value for ('ingott4', 'faction_armaments08_t5') is 89
Using existing node value for le

Using existing node value for timbert4: 92
New edge value for ('timbert4', '2hstafflife_graspinglightt4') is 92
Using existing node value for ingott4: 89
New edge value for ('ingott4', '2hstafflife_graspinglightt4') is 89
Using existing node value for alchemylifet3: 3
New edge value for ('alchemylifet3', '2hstafflife_graspinglightt4') is 6
Using existing node value for blessedcruciblet1: 3
New edge value for ('blessedcruciblet1', '2hstafflife_graspinglightt4') is 9
Using existing node value for scalecordt1: 3
New edge value for ('scalecordt1', '2hstafflife_graspinglightt4') is 9
New node value for 2hstafflife_graspinglightt4 is 243
2hstafflife_gortanstaintedstafft5
Using existing node value for timbert51: 1818
New edge value for ('timbert51', '2hstafflife_gortanstaintedstafft5') is 5454
Using existing node value for essencelifet1: 6
New edge value for ('essencelifet1', '2hstafflife_gortanstaintedstafft5') is 18
Using existing node value for blockt51: 1629
New edge value for ('blockt51'

New edge value for ('leathert4', '1hthrowingaxe_archmagesassistantt4') is 137
Using existing node value for blessedcruciblet1: 3
New edge value for ('blessedcruciblet1', '1hthrowingaxe_archmagesassistantt4') is 9
Using existing node value for brightscalehidet1: 3
New edge value for ('brightscalehidet1', '1hthrowingaxe_archmagesassistantt4') is 9
New node value for 1hthrowingaxe_archmagesassistantt4 is 374
1hswordt1
Using existing node value for rawhidet1: 3
New edge value for ('rawhidet1', '1hswordt1') is 3
Using existing node value for woodt1: 3
New edge value for ('woodt1', '1hswordt1') is 3
New node value for 1hswordt1 is 6
1hskinningknifet1
Using existing node value for flintt1: 3
New edge value for ('flintt1', '1hskinningknifet1') is 3
Using existing node value for woodt1: 3
New edge value for ('woodt1', '1hskinningknifet1') is 3
New node value for 1hskinningknifet1 is 6
1hsicklet1
Using existing node value for flintt1: 3
New edge value for ('flintt1', '1hsicklet1') is 3
Using exi

In [10]:
nodes = []
no_value_nodes = []
for node in F.nodes(data=True):
    if 'Value' in node[1]:
        nodes.append((node[1]['Value'], node[0]))
    else:
        no_value_nodes.append((node[0]))
nodes = sorted(nodes)  
no_value_nodes = sorted(no_value_nodes)
print("Nodes: ", len(nodes), nodes)
print("\n\n\n\n no_value_nodes: ", len(no_value_nodes))

Nodes:  3529 [(1, '0203_0103_hopecatalyst'), (1, '0203_0103_menderscloth'), (1, '0203_0103_mercysbreath'), (1, '0203_0103_soothsayersalve'), (1, '0204_0104_bladebrute'), (1, '0204_0104_bucaneersplank'), (1, '0204_0104_hangmansbinding'), (1, '0204_0104_infernalcoagulant'), (1, '0205_0102_attunedstarmetal'), (1, '0205_0102_baneseedoil'), (1, '0205_0102_essencepower'), (1, '0205_0102_evershinepolish'), (1, '0205_0102_flexiblemesh'), (1, '0205_0102_hammeredmetal'), (1, '0205_0102_heavypadding'), (1, '0205_0102_metalbanding'), (1, '0205_0102_refinedbalefire'), (1, '0205_0102_reforgedplating'), (1, '0205_0102_ruggedvarnish'), (1, '0205_0102_shardsbattle'), (1, '0205_0102_stalwartrivets'), (1, '0205_0102_stouthides'), (1, '0205_0102_treatedhobnails'), (1, '1002_0104_blightbanepitch'), (1, '1002_0104_corruptedthread'), (1, '1002_0104_duskfalldye'), (1, '1002_0104_essenceofshadow'), (1, '1002_0104_huntersclasps'), (1, '1002_0104_layeredleather'), (1, '1002_0104_muffledpadding'), (1, '1002_0104_

## Save our edge and label lists again

In [45]:
for n in F.predecessors('mediumhands_voidbentt5'):
    print(F.nodes['mediumhands_voidbentt5'])
    print(F.edges[n, 'mediumhands_voidbentt5'])
# F.edges['mediumhands_voidbentt5','tannint5']

{'Rarity': 4, 'Value': 17629}
{'Quantity': 3, 'Tradeskill': 'Armoring', 'Rarity': 4, 'Station': 'Outfitting Station Tier 5', 'ItemType': 'Armor', 'Category': 'Armor', 'Value': 5463}
{'Rarity': 4, 'Value': 17629}
{'Quantity': 3, 'Tradeskill': 'Armoring', 'Rarity': 4, 'Station': 'Outfitting Station Tier 5', 'ItemType': 'Armor', 'Category': 'Armor', 'Value': 5589}
{'Rarity': 4, 'Value': 17629}
{'Quantity': 3, 'Tradeskill': 'Armoring', 'Rarity': 4, 'Station': 'Outfitting Station Tier 5', 'ItemType': 'Armor', 'Category': 'Armor', 'Value': 5589}
{'Rarity': 4, 'Value': 17629}
{'Quantity': 3, 'Tradeskill': 'Armoring', 'Rarity': 4, 'Station': 'Outfitting Station Tier 5', 'ItemType': 'Armor', 'Category': 'Armor', 'Value': 729}
{'Rarity': 4, 'Value': 17629}
{'Quantity': 4, 'Tradeskill': 'Armoring', 'Rarity': 4, 'Station': 'Outfitting Station Tier 5', 'ItemType': 'Armor', 'Category': 'Armor', 'Value': 4}
{'Rarity': 4, 'Value': 17629}
{'Quantity': 3, 'Tradeskill': 'Armoring', 'Rarity': 4, 'Station'

In [52]:
now = datetime.now()

def GenerateGephiData(F):
    """
        ing?: ingredient list in the form of [xid, name, rarity, typ, itemType, icon, quantity]
    """
    fileString = "data/%s" % (now.strftime("%m.%d.%Y.%H.%M.%SVALUE_EDGELIST.csv"))
    with open(fileString, 'w') as csvfile:
        writer = csv.writer(csvfile)
        #writer.writeheader()
        writer.writerow(["Source", "Target", "Quantity", "Tradeskill", "Rarity", "Station", "ItemType", "Category", "Value"]) #These column headers are used in Gephi automatically
        for s,t,d in F.edges(data=True):
#             print("Key: ", key)
#             print("Value: ", value)
            tradeskill = d['Tradeskill']
            rarity = d['Rarity']
            station = d['Station']
            quantity = d['Quantity']
            item_type = d['ItemType']
            category = d['Category']
            value = d['Value']
#                 print(nodeA, nodeB, str(value), tradeskill, str(rarity), type(station), item_type)
#             print(s + " -> " + t + " " + "tradeskill: {} rarity: {} station: {} item_type: {}".format(tradeskill, str(rarity), type(station), item_type))
            print("Value: {}".format(value))
            writer.writerow([s, t, quantity, tradeskill, rarity, station, item_type, category, value]) #nodeA is ingredient, nodeB is recipe output, and count is the quantity


#Generates a new csv file for the node list labels on Gephi
def GenerateGephiLabels(F):
    fileString = "data/%s" % (now.strftime("%m.%d.%Y.%H.%M.%SVALUE_LABELS.csv"))
    print("Generating Labels...")
    sys.stdout.flush()
    with open(fileString, 'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["ID", "Label", "Rarity", "Value"]) #These columns are used in Gephi automatically
        for node, data in F.nodes(data=True):
#             print("Node: {} Data: {}".format(node, data))
            name = data['name'] if 'name' in data else node
            print(name)
            writer.writerow([node, name, data['Rarity'], data['Value']]) #This data is streamer1, streamer1, and # of unique viewers for streamer1
                
                
   

In [53]:
GenerateGephiData(F)
GenerateGephiLabels(F)

Value: 243
Value: 81
Value: 27
Value: 9
Value: 486
Value: 486
Value: 486
Value: 486
Value: 486
Value: 243
Value: 243
Value: 243
Value: 243
Value: 243
Value: 81
Value: 81
Value: 81
Value: 81
Value: 81
Value: 81
Value: 81
Value: 81
Value: 81
Value: 81
Value: 27
Value: 27
Value: 27
Value: 27
Value: 27
Value: 27
Value: 27
Value: 27
Value: 27
Value: 27
Value: 1458
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 162
Value: 54
Value: 54
Value: 54
Value: 54
Value: 54
Value: 54
Value: 54
Value: 54
Value: 54
Value: 54
Value: 54
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 3
Value: 1
Value: 1
Value: 1
Value: 1
Value: 5463
Value: 5463
Value: 5463
Value: 5463
Value: 5463
Value: 5463
Valu

Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 89
Value: 137
Value: 137
Value: 137
Value: 137
Value: 137
Value: 137
Value: 137
Value: 137
Value: 137
Value: 137
Value: 137
Value: 137
Value: 137
Value: 137
Value: 137
Value: 137
Value: 137
Value: 4
Value: 4
Value: 4
Value: 4
Value: 4
Value: 4
Value: 4
Value: 4
Value: 4
Value: 4
Value: 4
Value: 4
Value: 4
Value: 4
Value: 

Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
V

Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 70
Value: 70
Value: 70
Value: 11
Value: 11
Value: 11
Value: 3
Value: 3
Value: 3
Value: 480
Value: 480
Value: 480
Value: 70
Value: 70
Value: 70
Value: 11
Value: 11
Value: 11
Value: 3
Value: 3
Value: 480
Value: 480
Value: 480
Value: 70
Value: 70
Value: 70
Value: 11
Value: 11
Value: 11
Value: 3
Value: 3
Value: 243
Value: 243
Value: 243
Value: 54
Value: 54
Value: 54
Value: 9
Value: 9
Value: 9
Value: 3
Value: 3
Value: 3
Value: 480
Value: 480
Value: 480
Value: 70
Value: 70
Value: 70
Value: 11
Value: 11
Value: 11
Value: 3
Value: 3
Value: 480
Value: 480
Value: 480
Value: 

Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
Value: 1
V

...

Leathers
Tier 4 Metals
Tier 3 Metals
Tier 2 Metals
Tier 5 Protective Reagents
Water Reagents
Tier 4 Protective Reagents
Tier 3 Protective Reagents
Tier 2 Protective Reagents
Air Reagents
Life Arcana
Wood
Fire Arcana
Tier 5 Timber
Metals
Tier 4 Timber
Tier 3 Timber
Tier 2 Timber
Tier 5 Cloth
Tier 4 Cloth
Tier 3 Cloth
Cut Pristine Topaz
Trinket Bands
Trinket Gem Settings
Precious Metals
Cut Brilliant Topaz
Cut Topaz
Cut Flawed Topaz
Cut Pristine Sapphire
Cut Brilliant Sapphire
Cut Sapphire
Cut Flawed Sapphire
Cut Pristine Ruby
Cut Brilliant Ruby
Cut Ruby
Cut Flawed Ruby
Pristine Pearl
Brilliant Pearl
Pearl
Flawed Pearl
Cut Pristine Opal
Cut Brilliant Opal
Cut Opal
Cut Flawed Opal
Cut Pristine Onyx
Cut Brilliant Onyx
Cut Onyx
Cut Flawed Onyx
Cut Pristine Moonstone
Cut Brilliant Moonstone
Cut Moonstone
Cut Flawed Moonstone
Cut Pristine Malachite
Cut Brilliant Malachite
Cut Malachite
Cut Flawed Malachite
Cut Pristine Jasper
Cut Brilliant Jasper
Cut Jasper
Cut Flawed Jasper
Cut Pristine Emer

Minor Mining Gathering Trophy
Loaded Dice
Basic Loot Luck Trophy
Stacked Deck
Minor Loot Luck Trophy
Rabbit's Foot
Pure Resin
Basic Logging Gathering Trophy
Lumberjack's Token
Minor Logging Gathering Trophy
Mercurial Token
Basic Harvesting Gathering Trophy
Journal of Aeternum Flora
Minor Harvesting Gathering Trophy
Taxidermied Daemonaja
Basic Fishing Gathering Trophy
Taxidermied Blue-blooded Barb
Minor Fishing Gathering Trophy
Precision Engineering Tools
Basic Engineering Crafting Trophy
Engineer's Technique Manuals
Minor Engineering Crafting Trophy
Ephemeral Seal
Basic Lost Combat Trophy
Ectoplasmic Essence
Minor Lost Combat Trophy
Corrupted Totem
Basic Corrupted Combat Trophy
Corrupted Crest
Minor Corrupted Combat Trophy
Pristine Bear Claw
Basic Wildlife Combat Trophy
Pristine Wolf Claw
Minor Wildlife Combat Trophy
Glowing Sap
Basic Angry Earth Combat Trophy
Barkflesh
Minor Angry Earth Combat Trophy
Ancient Mandible
Basic Ancients Combat Trophy
Ancient Femur
Minor Ancients Combat Tro

Pristine Amber
A recipe for plant food
Earthspine Leaf
Dragonglory Flower
Fungal Spores
Shocking Lodestone
Putrid Bark
Metallic Boneweave
Jade Talisman
Corrupted Treatise
Vial of Ancient Mutagen
Vial Of Ectoplasmic Mutagen
Glittering Ebony
Empowered Counterbalance
Scaling Powder
Azurite Chunk
Shifthide
Blessed Crucible
Brightscale Hide
Fae Iron
Voidmetal
Life Crystal Core
Scalecord
Blessed Scepter
Glintstrands
Fire Crystal Core
Petrified Wood
Whisperwood
Petrified Fire
Ebonwood
Quillbark
Embossed Wrapping
Withered Bough
Intricate Firearm Lock
Fragmented Musket
Ancient Bearing
Enchanted Bow String
Splintered Bow
Broken Hatchet
Emblem of Fortitude
Etched Handguard
Serpentine Handguard
Shattered Sword
Crest of Fortitude
Ice Crystal Core
Eternal Ice
Recipe: Volusus Perfume
Slimy Fungal Drippings
Stag Anus Gland
Maggot-Laden Fruit
Sliver of Adamant
Coarse Sandpaper
Sand Flux
Tannin
Phasefiber
Shelldust Flux
Scaly Hide
Toadpot
Bumbleblossom
Cod Eye
Earthshell Tail
Platecap Flesh
Petalcap
Spi

lighthead_syndicaterank3_occultist
lighthead_syndicaterank3_cleric
lighthead_syndicaterank3_brigand
lighthead_syndicaterank3_barbarian
lighthead_syndicaterank2_occultist
lighthead_syndicaterank2_cleric
lighthead_syndicaterank2_brigand
lighthead_syndicaterank2_barbarian
lighthead_syndicaterank1_occultist
lighthead_syndicaterank1_cleric
lighthead_syndicaterank1_brigand
lighthead_syndicaterank1_barbarian
lighthead_shadewalkert5
lighthead_protectivewyrdhatt5
lighthead_marauderrank5_occultist
lighthead_marauderrank5_cleric
lighthead_marauderrank5_brigand
lighthead_marauderrank5_barbarian
lighthead_marauderrank4_occultist
lighthead_marauderrank4_cleric
lighthead_marauderrank4_brigand
lighthead_marauderrank4_barbarian
lighthead_marauderrank3_occultist
lighthead_marauderrank3_cleric
lighthead_marauderrank3_brigand
lighthead_marauderrank3_barbarian
lighthead_marauderrank2_occultist
lighthead_marauderrank2_cleric
lighthead_marauderrank2_brigand
lighthead_marauderrank2_barbarian
lighthead_maraude

heavylegs_syndicaterank5_cleric
heavylegs_syndicaterank5_brigand
heavylegs_syndicaterank4_occultist
heavylegs_syndicaterank4_cleric
heavylegs_syndicaterank4_brigand
heavylegs_syndicaterank3_occultist
heavylegs_syndicaterank3_cleric
heavylegs_syndicaterank3_brigand
heavylegs_syndicaterank2_occultist
heavylegs_syndicaterank2_cleric
heavylegs_syndicaterank2_brigand
heavylegs_syndicaterank1_occultist
heavylegs_syndicaterank1_cleric
heavylegs_syndicaterank1_brigand
heavylegs_marauderrank5_occultist
heavylegs_marauderrank5_cleric
heavylegs_marauderrank5_brigand
heavylegs_marauderrank4_occultist
heavylegs_marauderrank4_cleric
heavylegs_marauderrank4_brigand
heavylegs_marauderrank3_occultist
heavylegs_marauderrank3_cleric
heavylegs_marauderrank3_brigand
heavylegs_marauderrank2_occultist
heavylegs_marauderrank2_cleric
heavylegs_marauderrank2_brigand
heavylegs_marauderrank1_occultist
heavylegs_marauderrank1_cleric
heavylegs_marauderrank1_brigand
heavylegs_marauderlegatust5_occultist
heavylegs_ma

In [15]:
# resource_names = {'oret': 1,
#                   'ingott': 1, 
#                   'oreprecioust': 1,
#                   'ingotprecioust':1,
#                   'woodt':1,
#                   'timbert':1,
#                   'fibert':1,
#                   'clotht':1, 
#                   'woodt':1,
#                   'rawhidet':1,
#                   'leathert':1,}
# for r in resource_names.keys():
#     for i in range(0,7):
#         rid = r + '{}'.format(i)
#         try:
#             resource_data = F.node[rid]
# #             print("ID: {} Data: {}".format(rid, resource_data))
# #             print(" Value: {}".format(values[rid]))
#         except:
# #             print("WTF I DON'T EXIST {}".format(rid))
#             pass

In [16]:
# ## Uncomment if need a way that "works?", older and less reliable way of computing able.

# def compute_edge_value(F, node_values, source, target):  
#     value = 0
#     pdata = F.node[source]
#     edgedata = F.get_edge_data(source, target)
#     if source in node_values: 
#         start = source in start_items
#         value = node_values[source]
#     else:
#         value = compute_node_value(source, pdata, node_values, F) * float(edgedata['Quantity'])
#         F.add_node(source, value=value)
#         node_values[source] = value 
#     return value

# def compute_node_value(target, data, node_values, F):
#     print("Computing value of {}".format(target))
#     start = target in start_items
#     if target in node_values:
#         print("Already computed {} value of {}. Start? {} ".format(target, node_values[target], start))
#         value = node_values[target]
#     else:
#         evalues = []
#         print("Predecessors (ingredients) to {} are: {} {}".format(target, len(list(F.predecessors(target))), list(F.predecessors(target))))
#         for p in F.predecessors(target):
#             ev = compute_edge_value(F, node_values, p, target)
#             edge_values[(p, target)] = ev
#             evalues.append(ev)
#             print("p: {} evalues: {}".format(p, evalues))
#         value = sum(evalues)
#     F.add_node(target, value=value)
#     node_values[target] = value
#     print("New computed value for {} is {}".format(target, value))
#     for s in F.successors(target):
#         ev = compute_edge_value(F, node_values, target, s)
#         edge_values[(target, s)] = ev
#     print("Completed successor run for {}".format(target))
#     return value
            

# for ingId, data in F.nodes.data():
#     compute_node_value(ingId, data, node_values, F)   
    
# # iid = '2hgreataxe_darkskyt5'
# # # iid = 'house_housingitem_buff_skinning_bonus_t2'
# # data = F.node[iid]
# # if iid in node_values:
# #     del node_values[iid]
# # compute_node_value(iid, data, node_values, F)

